In [132]:
import tokenizer
from tqdm import tqdm
import re
from collections import Counter

In [4]:
def corpus_common_tokens(corpus, n=3*1e5):
    counter = Counter()
    for text in corpus:
        tokens = re.findall(r"\w+|[^\w\s]", text)
        counter.update(tokens)
    tokens, counts = zip(*counter.most_common(int(n)))
    return tokens


common_tokens = corpus_common_tokens(tokenizer.corpus)
tokenizer.test_tokenizer_from_corpus_fn(corpus_common_tokens)

In [5]:
class Tokenizer:
    
    def __init__(self, token_list): # token_list: List[{“piece”:str, ”id”:int}]
        self.token_list = token_list
        self.token_dict = {d['id']: d['piece'] for d in token_list}
        self.reverse_token_dict = {v: k for k,v in self.token_dict.items()}
        self.unk = '[UNK]'
        self.unk_id = 3
        
    def decode(self, ids):
        return ''.join([self.token_dict.get(id_, self.unk) for id_ in ids])
    
    def tokenize(self, string):
        tokens = re.findall(r"\w+|[^\w\s]", string)
        return [self.reverse_token_dict.get(token, self.unk_id) for token in tokens]
        
tokenizer.test_tokenizer(Tokenizer)

In [133]:
class BPETokenizer(Tokenizer):
    
    def __init__(self, token_list, k=int(1e3)):
        super().__init__(token_list)
        self.k = k
        
    def tokenize(self, string):
        merges, tokens = 0, list(string)
        while merges <= self.k:
            for idx, token in enumerate(tokens):
                # Oh we should merge idx and idx+1!
                any_merges=False
                if idx+1 < len(tokens):
                    print(idx, len(tokens))
                    if self.reverse_token_dict.get(token + tokens[idx+1], False):
                        merge_temp = token + tokens[idx+1]
                        tokens[idx] = merge_temp
                        tokens.pop(idx+1)
                        merges += 1
                        any_merges = True
                        break
            if not any_merges: break
        return tokens
    
    #Make a method from_corpus(corpus: List[str]) on your BPETokenizer class. 
    # It should break the texts into char tokens, 
    # then count the number of each pair of sequential token(char) in the corpus. 
    # Then create a new token that’s the concatenation of the two most common sequential tokens. 
    # Repeat this some number of times. 

    def from_corpus_old(self, corpus):
        tokens = [char for string in corpus for char in string]
        new_tokens = ['  ']  # ' ', ' ' -> '  '
        
        for i in range(self.k):
            pairs = [tokens[i-1] + token for i, token in enumerate(tokens, 1)]
            counter = Counter(pairs)
            new_token, _ = counter.most_common(1)[0]
            
            
    def from_corpus(self, corpus):
        tokens = [char for string in corpus for char in string]

        for i in tqdm(range(self.k)):
            pairs = [token + tokens[i+1] for i, token in enumerate(tokens[:-1])]
            
            d = {}
            for i, pair in enumerate(pairs):
                if pair not in d: d[pair] = []
                d[pair].append(i)
                
            most_common, most_common_i = max(d.items(), key=lambda pair: len(pair[1]))
            
            j, new_tokens = 0, []
            while j < len(tokens):
                if j in most_common_i: 
                    new_tokens.append(most_common)
                    j += 1
                else: new_tokens.append(tokens[j])
                j += 1
            
            tokens = new_tokens
            
        return tokens

---

# Viterbi pseudocode

- No substring of a string can be less likely than the string itself


```

```

In [144]:
reference = tokenizer.Tokenizer.from_corpus(tokenizer.corpus)
t = BPETokenizer(token_list=reference.token_list, k = 5000)

In [145]:
tokens = t.from_corpus(tokenizer.minicorpus)

100%|██████████| 5000/5000 [02:53<00:00, 28.80it/s]


In [159]:
max([len(t) for t in tokens])

11829

In [147]:
set([t for t in tokens if len(t) >= 10])

{'\n\n\n\n\nACT IV',
 '\n\n                    ',
 '\n    I am ',
 '\n    Must ',
 '\n    Than ',
 '\n    That ',
 '\n    This ',
 '\n    Till ',
 '\n    To give ',
 '\n    We have ',
 '\n    What ',
 '\n    Which ',
 '\n    displeasure',
 '\n    of the ',
 '\n    your ',
 '\n  BERTRAM. ',
 '\n  PAROLLES. ',
 '              ',
 '                                ',
 '      Exeun',
 "  PAROLLES. O, let me live,\n    And all the secrets of our camp I'll show,\n    Their force, their purposes. Nay, I'll speak that\n    Which you will wonder at.\n  FIRST SOLDIER. But wilt thou faithfully?\n  PAROLLES. If I do not, damn me.\n  FIRST SOLDIER. Acordo linta.\n    Come on; thou art granted space.\n                   Exit, PAROLLES guarded. A short alarum within\n  SECOND LORD. Go, tell the Count Rousillon and my brother\n    We have caught the woodcock, and will keep him muffled\n    Till we do hear from them.\n  SECOND SOLDIER. Captain, I will.\n  SECOND LORD. 'A will betray us all unto ourselve

In [73]:
import pdb

def key(k):
    pdb.set_trace()
    
    

max(d.items(), key=lambda pair: len(pair[1]))

--Return--
> <ipython-input-73-5c998fd87cb8>(4)key()->None
-> pdb.set_trace()


(Pdb)  k


('  ', [0, 28, 29, 30, 75, 76, 77, 129, 130, 131, 159, 202, 236, 267, 268, 269, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 369, 428, 429, 430, 487, 488, 489, 518, 553, 606, 607, 608, 626, 660, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 863, 917, 951, 978, 979, 980, 1027, 1028, 1029, 1072, 1073, 1074, 1124, 1125, 1126, 1163, 1164, 1165, 1211, 1212, 1213, 1259, 1260, 1261, 1304, 1305, 1306, 1338, 1368, 1397, 1410, 1411, 1412, 1453, 1454, 1455, 1482, 1508, 1511, 1512, 1513, 1556, 1557, 1558, 1599, 1600, 1601, 1653, 1654, 1655, 1688, 1717, 1718, 1719, 1768, 1769, 1770, 1820, 1821, 1822, 1855, 1884, 1939, 1940, 1941, 1988, 1989, 1990, 2032, 2033, 2034, 2094, 2095, 2096, 2143, 2144, 2145, 2194, 2195, 2196, 2

(Pdb)  q


BdbQuit: 

In [38]:
counter.most_common(1)[0]

('  ', 10896)

In [21]:
corpus = tokenizer.minicorpus
tokens = [char for string in corpus for char in string]


In [14]:
t.from_corpus(corpus)

SyntaxError: iterable unpacking cannot be used in comprehension (<ipython-input-14-b5bcae41da3c>, line 1)

In [16]:
t.tokenize("I am a chicken!")

0 15
1 15
2 15
0 14
1 14
2 14
3 14
4 14
5 14
6 14
7 14
8 14
9 14
10 14
11 14
0 13
1 13
2 13
3 13
4 13
5 13
6 13
7 13
8 13
9 13
10 13
0 12
1 12
2 12
3 12
4 12
5 12
6 12
7 12
8 12
9 12
10 12


['I', ' ', 'am', ' ', 'a', ' ', 'c', 'h', 'i', 'c', 'ken', '!']

In [20]:
t.tokenize("I can do spit today")

0 19
1 19
2 19
3 19
0 18
1 18
2 18
0 17
1 17
2 17
3 17
4 17
0 16
1 16
2 16
3 16
4 16
5 16
6 16
7 16
8 16
0 15
1 15
2 15
3 15
4 15
5 15
6 15
7 15
0 14
1 14
2 14
3 14
4 14
5 14
6 14
0 13
1 13
2 13
3 13
4 13
5 13
6 13
7 13
8 13
0 12
1 12
2 12
3 12
4 12
5 12
6 12
7 12
8 12
0 11
1 11
2 11
3 11
4 11
5 11
6 11
7 11
8 11
9 11
0 10
1 10
2 10
3 10
4 10
5 10
6 10
7 10
8 10
0 9
1 9
2 9
3 9
4 9
5 9
6 9
7 9


['I', ' ', 'can', ' ', 'do', ' ', 'spit', ' ', 'today']

In [2]:
import json

known_dist = {}
with open('sentencepiece.json', 'r') as f:
    known_dist = json.load(f)


def viterbi_this(known_dist, sentence):
    # Do Viterbi things
    # 1. count up the possible tokenization permutation for the sentence
    tokens = [char for string in corpus for char in sentence]
    
    token_v_score = [(char, score) for char in tokens]
    #loop up on a
    
    # 2. See which permutation has the lowest entropy
    # 3. use said "lowest entropy" permutation
    

In [6]:
known_dist.sort(key = lambda x: -x['score'])

In [7]:
known_dist

[{'piece': '<pad>', 'id': 0, 'score': 0.0, 'type': 3},
 {'piece': '<unk>', 'id': 1, 'score': 0.0, 'type': 2},
 {'piece': '[CLS]', 'id': 2, 'score': 0.0, 'type': 3},
 {'piece': '[SEP]', 'id': 3, 'score': 0.0, 'type': 3},
 {'piece': '[MASK]', 'id': 4, 'score': 0.0, 'type': 3},
 {'piece': '(', 'id': 5, 'score': 0.0, 'type': 4},
 {'piece': ')', 'id': 6, 'score': 0.0, 'type': 4},
 {'piece': '"', 'id': 7, 'score': 0.0, 'type': 4},
 {'piece': '-', 'id': 8, 'score': 0.0, 'type': 4},
 {'piece': '.', 'id': 9, 'score': 0.0, 'type': 4},
 {'piece': '–', 'id': 10, 'score': 0.0, 'type': 4},
 {'piece': '£', 'id': 11, 'score': 0.0, 'type': 4},
 {'piece': '€', 'id': 12, 'score': 0.0, 'type': 4},
 {'piece': '▁', 'id': 13, 'score': -2.1889710426330566, 'type': 1},
 {'piece': '▁the', 'id': 14, 'score': -3.07045841217041, 'type': 1},
 {'piece': ',', 'id': 15, 'score': -3.1902313232421875, 'type': 1},
 {'piece': '▁of', 'id': 16, 'score': -3.8403830528259277, 'type': 1},
 {'piece': '▁and', 'id': 17, 'score': 